In [1]:
import tensorflow as tf

2023-08-14 10:32:49.338057: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-14 10:32:49.408748: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-14 10:32:49.410617: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-14 10:32:50.631615: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


<h3 style='color:purple'>Create tf dataset from a list</h3>

In [2]:
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

<h3 style='color:purple'>Iterate through tf dataset</h3>

In [3]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


<h3 style='color:purple'>Iterate through elements as numpy elements</h3>

In [4]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


<h3 style='color:purple'>Iterate through first n elements in tf dataset</h3>

In [5]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


<h3 style='color:purple'>Filter sales numbers that are < 0</h3>

In [6]:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


In [7]:
# custom print statments
def pp(data):
    list = []
    for i in data.as_numpy_iterator():
        list.append(i)
    return list

<h3 style='color:purple'>Convert sales numbers from USA dollars ($) to Indian Rupees (INR) Assuming 1->72 conversation rate</h3>

In [8]:
tf_dataset = tf_dataset.map(lambda x: x*72)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2448
2232


<h3 style='color:purple'>Shuffe</h3>

In [9]:
tf_dataset = tf_dataset.shuffle(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
2232
1584
2448
2304
2232


https://stackoverflow.com/questions/53514495/what-does-batch-repeat-and-shuffle-do-with-tensorflow-dataset

<h3 style='color:purple'>Batching</h3>

In [10]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[1512 2232]
[2304 1584]
[2448 2232]


<h3 style='color:purple'>Perform all of the above operations in one shot</h3>

In [11]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*72).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1584 2232]
[1512 2304]
[2448 2232]


<h3>Images</h3>

In [12]:
images_ds = tf.data.Dataset.list_files('images/*/*', shuffle=False)

In [13]:
image_count = len(images_ds)
image_count

130

In [14]:
type(images_ds)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [15]:
for file in images_ds.take(3):
    print(file.numpy())

b'images/cat/20 Reasons Why Cats Make the Best Pets....jpg'
b'images/cat/7 Foods Your Cat Can_t Eat.jpg'
b'images/cat/A cat appears to have caught the....jpg'


In [16]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3):
    print(file.numpy())

b'images/dog/66 gifts for dogs or dog lovers to get_yythk....jpg'
b'images/dog/8 amazing Indian dog breeds that....png'
b'images/dog/List of Dog Breeds _ Petfinder.jpg'


In [17]:
class_names = ["cat","dog"]

In [18]:
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [19]:
len(train_ds)

104

In [20]:
len(test_ds)

26

In [21]:
def get_label(file_path):
    import os
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

In [23]:
get_label("images/dog/66 gifts for dogs or dog lovers to get_yythk....jpg")

<tf.Tensor: shape=(), dtype=string, numpy=b'dog'>

In [24]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path) # read the raw data from the file as a string
    img = tf.image.decode_jpeg(img, channels=3) # convert the compressed string to a 3D uint8 tensor
    img = tf.image.resize(img,[128,128])
    return img, label

In [25]:
img, label = process_image("images/dog/66 gifts for dogs or dog lovers to get_yythk....jpg")
img.numpy()[:2]

array([[[152.65662 , 131.65662 , 110.65662 ],
        [148.89807 , 127.89807 , 106.89807 ],
        [156.20251 , 133.85876 , 115.53064 ],
        [163.85156 , 142.85156 , 123.85156 ],
        [163.54016 , 144.54016 , 127.54016 ],
        [166.      , 149.      , 133.      ],
        [169.54688 , 152.54688 , 136.54688 ],
        [171.      , 155.      , 142.      ],
        [173.49634 , 157.49634 , 144.49634 ],
        [172.      , 160.      , 148.      ],
        [173.      , 161.      , 149.      ],
        [ 39.247192,  37.684692,  24.684692],
        [178.64062 , 165.64062 , 156.64062 ],
        [177.59045 , 167.59045 , 157.59045 ],
        [179.      , 169.      , 160.      ],
        [180.      , 170.      , 161.      ],
        [181.      , 172.      , 163.      ],
        [182.38281 , 173.38281 , 164.38281 ],
        [185.      , 176.      , 167.      ],
        [185.2218  , 178.2218  , 168.2218  ],
        [185.57251 , 175.90063 , 165.90063 ],
        [183.38281 , 176.22656 , 1

In [26]:
train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [27]:
for image, label in train_ds.take(1):
    print("****",image)
    print("****",label)

**** tf.Tensor(
[[[214.98438  182.70312  167.14062 ]
  [211.98438  183.98438  169.98438 ]
  [214.2461   186.2461   172.2461  ]
  ...
  [133.08594  139.08594   95.08594 ]
  [129.73828  135.73828   87.73828 ]
  [128.28015  135.28015   84.28015 ]]

 [[216.       184.28125  168.4375  ]
  [216.51746  188.51746  174.51746 ]
  [217.       189.       175.      ]
  ...
  [134.17664  140.17664   96.176636]
  [132.37708  138.37708   90.377075]
  [131.       138.        87.      ]]

 [[217.       188.       171.4375  ]
  [218.       189.       175.      ]
  [214.40625  186.40625  172.40625 ]
  ...
  [135.       141.        97.      ]
  [134.       140.        92.      ]
  [132.       139.        88.      ]]

 ...

 [[ 33.        39.        53.      ]
  [ 34.        40.        54.      ]
  [ 37.        40.        55.      ]
  ...
  [141.31262  148.31262  104.31262 ]
  [142.       149.       105.      ]
  [143.       150.       106.      ]]

 [[ 34.        40.        54.      ]
  [ 34.        40.   

In [28]:
def scale(image, label):
    return image/255, label

In [29]:
train_ds = train_ds.map(scale)

In [31]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [0.6059436  0.5353554  0.45692402]
****Label:  b'dog'
****Image:  [0.99607843 0.99607843 0.99607843]
****Label:  b'dog'
****Image:  [0.90226716 0.22889093 0.02723652]
****Label:  b'cat'
****Image:  [0.9588618  0.9602941  0.91736364]
****Label:  b'dog'
****Image:  [0.31072304 0.31072304 0.31072304]
****Label:  b'dog'
